In [1]:
from datetime import date
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 5)

df = catalog.load('raw/openaire/researchproduct_dev#parquet')

[09/08/25 12:32:22] INFO     Loading data from raw/openaire/researchproduct_dev#parquet         ]8;id=232034;file:///home/pablo/dev/conicet/kedro-conicet/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=68002;file:///home/pablo/dev/conicet/kedro-conicet/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\
                             (ParquetDataset)...                                                                   

In [2]:
def _pick_load_dt(df: pd.DataFrame):
    # Si hay una sola fecha en el batch, usala; si hay varias, quedate con la más reciente;
    # si no hay, hoy.
    if 'load_datetime' not in df.columns or df['load_datetime'].isna().all():
        return date.today()
    vals = df['load_datetime'].dropna()
    if vals.nunique() == 1:
        return vals.iloc[0]
    return pd.to_datetime(vals).max().date()

In [3]:
df_research_sources = df.loc[:,['id','sources']]
df_research_sources.dropna(inplace=True)

In [4]:
df_research_sources

,id,sources
0,4dc99724cf04::319dc88111c9b2d6021228590e79130a,[Revista de Investigaciones Veterinarias del P...
1,4dc99724cf04::95ea5df70a451a0487e051faa6c0a646,[Revista de Investigaciones Veterinarias del P...
...,...,...
298,dedup_wf_002::00cece367574590aa05deb123340c6e6,"[reponame:SEDICI (UNLP), instname:Universidad ..."
299,dedup_wf_002::00cf1fe88397a2c3ce9d41fe616b29cf,[Olivar: Revista de Literatura y Cultura Españ...


In [5]:
df_research_sources = df_research_sources.explode('sources').reset_index(drop=True)

In [6]:
df_research_sources

,id,sources
0,4dc99724cf04::319dc88111c9b2d6021228590e79130a,Revista de Investigaciones Veterinarias del Pe...
1,4dc99724cf04::319dc88111c9b2d6021228590e79130a,Revista de Investigaciones Veterinarias del Pe...
...,...,...
579,dedup_wf_002::00cf1fe88397a2c3ce9d41fe616b29cf,reponame:Memoria Académica (UNLP-FAHCE)
580,dedup_wf_002::00cf1fe88397a2c3ce9d41fe616b29cf,instname:Universidad Nacional de La Plata. Fac...


In [7]:
def openaire_land_researchproduct_sources(df: pd.DataFrame)-> pd.DataFrame:

    load_dt = _pick_load_dt(df)

    df_research_sources = df.loc[:,['id','sources']]
    df_research_sources.dropna(inplace=True)
    
    df_research_sources = df_research_sources.explode('sources').reset_index(drop=True)

    df_research_sources['load_datetime'] = load_dt

    return df_research_sources


In [8]:
df_research_sources = openaire_land_researchproduct_sources(df)

In [9]:
df_research_sources

,id,sources,load_datetime
0,4dc99724cf04::319dc88111c9b2d6021228590e79130a,Revista de Investigaciones Veterinarias del Pe...,2025-09-04
1,4dc99724cf04::319dc88111c9b2d6021228590e79130a,Revista de Investigaciones Veterinarias del Pe...,2025-09-04
...,...,...,...
579,dedup_wf_002::00cf1fe88397a2c3ce9d41fe616b29cf,reponame:Memoria Académica (UNLP-FAHCE),2025-09-04
580,dedup_wf_002::00cf1fe88397a2c3ce9d41fe616b29cf,instname:Universidad Nacional de La Plata. Fac...,2025-09-04


In [10]:
filter = df_research_sources['id'] == 'a9ac50f576aa::35dac175a7bd04110302c335589a676c'
df_research_sources[filter]

,id,sources,load_datetime


In [11]:
filter = df['id'] == 'a9ac50f576aa::35dac175a7bd04110302c335589a676c'
df[filter]

,authors,openAccessColor,publiclyFunded,type,language,countries,subjects,mainTitle,subTitle,descriptions,publicationDate,publisher,embargoEndDate,sources,formats,contributors,coverages,bestAccessRight,container,documentationUrls,codeRepositoryUrl,programmingLanguage,contactPeople,contactGroups,tools,size,version,geoLocations,id,originalIds,pids,dateOfCollection,lastUpdateTimeStamp,indicators,projects,organizations,communities,collectedFrom,instances,isGreen,isInDiamondJournal,relOrganizationId,load_datetime
24,"[{'fullName': 'Poujol, Lourdes María', 'name':...",None,False,publication,"{'code': 'eng', 'label': 'English'}",None,"[{'provenance': None, 'subject': {'scheme': 'k...",Expanding the analysis looks at infant mortali...,None,None,2011-01-01,None,None,None,None,"[Girardot, Jean-Jacques, /www.territorial-inte...",None,"{'code': 'c_14cb', 'label': 'CLOSED', 'scheme'...",None,None,None,None,None,None,None,None,None,None,a9ac50f576aa::35dac175a7bd04110302c335589a676c,[50|a9ac50f576aa::35dac175a7bd04110302c335589a...,None,None,None,"{'citationImpact': {'citationClass': 'C5', 'ci...",None,"[{'acronym': 'UNLP', 'id': 'openorgs____::40b9...",None,[{'key': 'openaire____::39e33d59918c2cb40d10ae...,"[{'accessRight': {'code': 'c_14cb', 'label': '...",False,False,openorgs____::40b9f835648a3e0d057d6917dd7e54d5,2025-09-04
